In [ ]:
! pip install super_gradients==3.0.0 
! pip install matplotlib==3.1.3 
from super_gradients import Trainer
from google.colab import files
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader
from super_gradients.training import models
from super_gradients.training import Trainer
from super_gradients.training import training_hyperparams
from super_gradients.training.losses import KDLogitsLoss, LabelSmoothingCrossEntropyLoss
from super_gradients.training import KDTrainer
import os
from PIL import Image
import numpy as np
import requests
from super_gradients.training.metrics import Accuracy, Top5

In [ ]:
! pip install -q kaggle
files.upload()
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d omarrelsayeed/plants-classification
!unzip /content/plants-classification.zip -d /content/alldataset

In [ ]:
experiment_name = "classification_transfer_learning"
model_name = "resnet50"

CHECKPOINT_DIR = '/notebook_ckpts/'
trainer = Trainer(experiment_name=experiment_name, ckpt_root_dir=CHECKPOINT_DIR)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Define transformations for training and test sets-
transform_train = transforms.Compose(
    [
        transforms.Resize((124,124)),
      transforms.ToTensor(),
     ]
     )

data_dir = 'My_Datasets/Cat_Dog_data/'

all_data = datasets.ImageFolder("/content/alldataset/All_dataset", 
                                  transform = transform_train)
train_set, val_set = torch.utils.data.random_split(all_data, [8000, 465])
classes = all_data.classes

train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True)
test_dataloader = DataLoader(val_set, batch_size=64, shuffle=True)

In [ ]:
classes

In [ ]:
####################### STUDENT ###################
model = models.get(model_name=model_name ,  num_classes=34, pretrained_weights="imagenet")
training_params =  training_hyperparams.get("training_hyperparams/cifar10_resnet_train_params")
training_params["max_epochs"] = 1
trainer.train(model=model, 
              training_params=training_params, 
              train_loader=train_dataloader,
              valid_loader=test_dataloader)

Downloading: "https://deci-pretrained-models.s3.amazonaws.com/KD_ResNet50_Beit_Base_ImageNet/resnet.pth" to /root/.cache/torch/hub/checkpoints/KD_ResNet50_Beit_Base_ImageNet_resnet.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/super_gradients/training/training_hyperparams/training_hyperparams.py:24: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(config_dir=pkg_resources.resource_filename("super_gradients.recipes", "")):
/usr/local/lib/python3.8/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'training_hyperparams/cifar10_resnet_train_params': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


sg_trainer -INFO- Started training for 1 epochs (0/0)



Train epoch 0:   0%|          | 0/125 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/cuda/memory.py:395: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(
Validation epoch 0: 100%|██████████| 8/8 [00:01<00:00,  4.60it/s]


SUMMARY OF EPOCH 0
├── Training
│   └── Labelsmoothingcrossentropyloss = 1.2383
└── Validation
    ├── Accuracy = 0.6151
    └── Labelsmoothingcrossentropyloss = 1.2528

base_sg_logger -INFO- Checkpoint saved in /notebook_ckpts/classification_transfer_learning/ckpt_best.pth
sg_trainer -INFO- Best checkpoint overriden: validation Accuracy: 0.6150537729263306
sg_trainer -INFO- RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...


Validation epoch 1: 100%|██████████| 8/8 [00:02<00:00,  3.70it/s]

SUMMARY OF EPOCH 1
├── Training
│   └── Labelsmoothingcrossentropyloss = 1.2383
└── Validation
    ├── Accuracy = 0.6151
    │   ├── Best until now = 0.6151 (↘ 0.0)
    │   └── Epoch N-1      = 0.6151 (↘ 0.0)
    └── Labelsmoothingcrossentropyloss = 1.2528
        ├── Best until now = 1.2528 (↘ -0.0)
        └── Epoch N-1      = 1.2528 (↘ -0.0)



In [ ]:
############## Teacher ####################
pretrained_beit = models.get('beit_base_patch16_224', arch_params={'num_classes': 34, "image_size": [224, 224], "patch_size": [16, 16]}, pretrained_weights="imagenet")
trainer2 = Trainer(experiment_name="beit_base_patch16_224_test", ckpt_root_dir="/notebook_ckpts/")
experiment_name = "kd_cifar10_resnet"
checkpoint_dir = "/notebook_ckpts/"
kd_trainer = KDTrainer(experiment_name=experiment_name, ckpt_root_dir=checkpoint_dir)


kd_params = {
    "max_epochs": 1,          # We will stop after 3 epochs because it is slow to train on google collab
    'lr_cooldown_epochs': 0,  # We dont want to use lr cooldown since we only train for 3 epochs
    'lr_warmup_epochs': 0,    # We dont want to use lr  warmup  since we only train for 3 epochs
    "loss": KDLogitsLoss(distillation_loss_coeff=0.8, task_loss_fn=LabelSmoothingCrossEntropyLoss()),
    "loss_logging_items_names": ["Loss", "Task Loss", "Distillation Loss"]}

training_params = training_hyperparams.get("imagenet_resnet50_kd", overriding_params=kd_params)
arch_params={"teacher_input_adapter": transforms.Resize(224)}

kd_trainer.train(training_params=training_params,
                 student=model,
                 teacher=pretrained_beit,
                 kd_architecture="kd_module",
                 kd_arch_params=arch_params,
                 train_loader=train_dataloader, valid_loader = test_dataloader)

In [ ]:
_ , acc , t5 = trainer.test(model=model, test_loader=test_dataloader, test_metrics_list=[Accuracy(), Top5()])

Test: 100%|██████████| 8/8 [00:01<00:00,  4.26it/s]


In [ ]:
acc , t5

(tensor(0.8366, device='cuda:0'), tensor(0.9699, device='cuda:0'))

In [ ]:
image = np.asarray(Image.open("/content/alldataset/All_dataset/Carolin/20221224_132919.jpg"))

pred_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

model.eval()
predictions = model(pred_transforms(image).unsqueeze(0).to('cuda'))

plt.xlabel(classes[torch.argmax(predictions)])
plt.imshow(image)

In [ ]:
onnx_filename = f"{experiment_name}_{model_name}.onnx"

model.eval()
model.prep_model_for_conversion(input_size=[1, 3, 32, 32]) # image shape of CIFAR10 images
dummy_input = torch.randn([1, 3, 32, 32], device=next(model.parameters()).device)

torch.onnx.export(model, dummy_input, onnx_filename)